In [1]:
import re
import os, os.path
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy
import pandas as pd
import warnings

# supress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas", lineno=570)

# custom tokenizer with stemmer 
class PorterTokenizer(object):
    def __init__(self):
        self.pt = PorterStemmer()
    def __call__(self, doc):
        return [self.pt.stem(t) for t in RegexpTokenizer(r'(?u)\b\w\w+\b').tokenize(doc)]

In [15]:
# import nltk
# nltk.download()

In [73]:
# process generated files
abs_path = r'/home/yurii/projects/mownit2/lab6/outl/AB/'
cur_path = r'/home/yurii/projects/mownit2/lab6/a/'
for filename in os.listdir(abs_path):
    with open(abs_path + filename, 'r') as myfile:
        s = myfile.read()
        p = re.compile('<doc[^>]*>([^<]*)<\/doc>')
        l = p.findall(s)
        for i in range(0,len(l)):
            p = re.compile('Notes\n|Footnotes\n|Bibliography\n|References\n|External links\n|Further reading\n')
            l[i] = p.sub(r'', l[i])
            p = re.compile('\n{3,}')
            l[i] = p.sub(r'\n', l[i])
            if l[i][0] == '\n':
                l[i] = l[i][1:]
            title = l[i].splitlines()[0]
            title = title.replace(r'/', ' ')
            with open(cur_path + title, 'w+') as myfile2:
                myfile2.write(l[i])

In [7]:
# read each file into separate string
art_path = r'/home/yurii/projects/mownit2/lab6/ex2/'
file_list = [name for name in os.listdir(art_path)]
doc_list = []
for file in file_list:
    with open(art_path + file, 'r') as myfile:
        doc_list.append(myfile.read())

8

In [68]:
# vectorize strings using tf-idf vectorizer
tfidf_matrix = TfidfVectorizer(min_df=1, stop_words='english', tokenizer=PorterTokenizer(), smooth_idf=False)
tfidf_trans_matrix = tfidf_matrix.fit_transform(doc_list)

In [48]:
# dt = pd.DataFrame(tfidf_trans_matrix.toarray(), index=file_list, columns=tfidf_trans_matrix.get_feature_names()).head(10)

In [69]:
search_str = ["blood guts beer"]

In [70]:
tfidf_search_matrix = TfidfVectorizer(vocabulary=tfidf_matrix.vocabulary_, stop_words='english', tokenizer=PorterTokenizer())
tfidf_trans_smatrix = tfidf_search_matrix.fit_transform(search_str)

/home/yurii/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(float(n_samples) / df) + 1.0


In [58]:
sim_matrix = cosine_similarity(tfidf_trans_matrix[0], tfidf_trans_smatrix)

In [57]:
tfidf_trans_matrix[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.00876051,  0.00876051]])

In [67]:
print(tfidf_trans_smatrix.toarray())

[[ 0.57735027  0.57735027  0.57735027]]


In [55]:
# print(sim_matrix)

In [60]:
sim_matrix = cosine_similarity(tfidf_trans_vect)
print(sim_matrix)

[[ 1.          0.05763932  0.03749549  0.15858565  0.11978858  0.02675291
   0.0907006   0.11164189]
 [ 0.05763932  1.          0.03148289  0.08993957  0.08264327  0.02422493
   0.06884484  0.07784266]
 [ 0.03749549  0.03148289  1.          0.04795767  0.06178963  0.49599109
   0.05040216  0.08425274]
 [ 0.15858565  0.08993957  0.04795767  1.          0.50723506  0.03950431
   0.19850558  0.17649297]
 [ 0.11978858  0.08264327  0.06178963  0.50723506  1.          0.03595162
   0.16197324  0.16648425]
 [ 0.02675291  0.02422493  0.49599109  0.03950431  0.03595162  1.
   0.04547475  0.04851268]
 [ 0.0907006   0.06884484  0.05040216  0.19850558  0.16197324  0.04547475
   1.          0.17426942]
 [ 0.11164189  0.07784266  0.08425274  0.17649297  0.16648425  0.04851268
   0.17426942  1.        ]]


In [21]:
# tfidf_trans_vect.toarray()

In [16]:
# vector = CountVectorizer(min_df=1, stop_words='english', tokenizer=PorterTokenizer())
# trans_vect = vector.fit_transform(doc_list)
# dt = pd.DataFrame(trans_vect.toarray(), index=file_list, columns=vector.get_feature_names()).head(10)

In [14]:
# vector.get_feature_names()

In [29]:
# example = [
#     'In mathematics, 1 − 2 + 3 − 4 + ··· is the infinite series whose terms are the successive positive integers, given alternating signs. Using sigma summation notation the sum of the first "m" terms of the series can be expressed as',
#     'The infinite series diverges, meaning that its sequence of partial sums, , does not tend towards any finite limit. Nonetheless, in the mid-18th century, Leonhard Euler wrote what he admitted to be a paradoxical equation:',
#     "A rigorous explanation of this equation would not arrive until much later. Starting in 1890, Ernesto Cesàro, Émile Borel and others investigated well-defined methods to assign generalized sums to divergent series—including new interpretations of Euler's attempts."
# ]

# vect1 = CountVectorizer(min_df=1, stop_words='english', tokenizer=PorterTokenizer())
# # vect2 = CountVectorizer(min_df=1, stop_words='english')
# dtm1 = vect1.fit_transform(example)
# # dtm2 = vect2.fit_transform(example)

In [ ]:
# TODO: IDF

In [32]:
# pd.DataFrame(dtm1.toarray(), index=example, columns=vect1.get_feature_names()).head(10)
# vect1.get_feature_names()

In [33]:
# pd.DataFrame(dtm2.toarray(), index=example, columns=vect2.get_feature_names()).head(10)
# vect2.get_feature_names()

In [12]:
# dict_list = [col.OrderedDict([]) for i in range(0,len(file_list))]

# def read_words(words_file):
#     return [word for line in open(words_file, 'r') for word in line.split()]

# file_name = art_path + r'0'
# wl = read_words(file_name)